<a href="https://colab.research.google.com/github/Shrey-Viradiya/NVIDIA_DALI_PyTorch_keras_tensorflow-Example/blob/main/NVIDIA_DALI_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Feb 27 19:08:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist nvidia-dali-cuda100
!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist nvidia-dali-tf-plugin-cuda100

Looking in indexes: https://pypi.org/simple, https://developer.download.nvidia.com/compute/redist
     |████████████████████████████████| 389.9MB 21kB/s 
Looking in indexes: https://pypi.org/simple, https://developer.download.nvidia.com/compute/redist
     |████████████████████████████████| 317kB 7.8MB/s 
  Created wheel for nvidia-dali-tf-plugin-cuda100: filename=nvidia_dali_tf_plugin_cuda100-0.31.0-cp37-cp37m-linux_x86_64.whl size=99370 sha256=3f888af871b09bc2e340f9c0c3a00e7163c5b3119ee9db9308fa4843f7695c4c
  Stored in directory: /root/.cache/pip/wheels/1a/ed/d9/b7adce71d3ed71de92958a4b1a30a7bfe5cd2003e21be04214
Successfully built nvidia-dali-tf-plugin-cuda100


### Shrey Viradiya
### 18BCE259

In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image_dataset_from_directory

import os
import scipy as sp
import matplotlib.pyplot as plt
import sklearn.metrics

from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
import nvidia.dali.types as types

import nvidia.dali.plugin.tf as dali_tf
import tensorflow as tf
import tensorflow.compat.v1 as tf_v1
import logging

In [4]:
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip \
  && unzip -qq flower_data.zip

In [5]:
train_directory = '/content/flower_data/train'
valid_directory = '/content/flower_data/valid'

In [6]:
!find "/content/flower_data/train/" -type f -print | wc -l
!find "/content/flower_data/valid/" -type f -print | wc -l

6552
818


In [7]:
class Data_Pipeline(Pipeline):
    def __init__(self, batch_size, device, data_path, device_id=0, num_threads=4, seed=0):
        super(Data_Pipeline, self).__init__(
            batch_size, num_threads, device_id, seed)
        self.device = device
        self.reader = ops.FileReader(file_root =data_path, random_shuffle=True)
        self.decode = ops.ImageDecoder(
            device='mixed' if device == 'gpu' else 'cpu',
            output_type=types.RGB)
        self.cmn = ops.CropMirrorNormalize(
            device=device,
            dtype=types.FLOAT,
            std=[255.],
            output_layout="HWC")
        self.rotate = ops.Rotate(device = "gpu")
        self.rng = ops.random.Uniform(range = (-10.0, 10.0))
        self.coin = ops.random.CoinFlip(probability = 0.5)
        self.flip = ops.Flip(device = "gpu")
        self.res = ops.Resize(device="gpu", resize_x=224, resize_y=224, interp_type=types.INTERP_TRIANGULAR)

    def define_graph(self):
        inputs, labels = self.reader(name="Reader")
        images = self.decode(inputs)
        if self.device == 'gpu':
            labels = labels.gpu()
        images = self.cmn(images)
        angle = self.rng()
        images = self.rotate(images, angle=angle)
        images = self.flip(images, horizontal = self.coin(), vertical = self.coin())
        images = self.res(images)
        return (images, labels)

In [8]:
BATCH_SIZE = 64
DROPOUT = 0.2
IMAGE_SIZE = 224
NUM_CLASSES = 102
HIDDEN_SIZE = 512
EPOCHS = 20
DATA_SIZE = 6552
VALIDATION_SIZE = 6552
ITERATIONS_PER_EPOCH = DATA_SIZE // BATCH_SIZE
VALIDATION_STEPS = VALIDATION_SIZE // BATCH_SIZE

shapes = (
    (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3),
    (BATCH_SIZE))
dtypes = (
    tf.float32,
    tf.int32)

In [9]:
Pipeline_train = Data_Pipeline(BATCH_SIZE, device='gpu', data_path = train_directory, device_id=0)
Pipeline_valid = Data_Pipeline(BATCH_SIZE, device='gpu', data_path = valid_directory, device_id=0)

In [10]:
with tf.device('/gpu:0'):
    data_set = dali_tf.DALIDataset(
        pipeline=Pipeline_train,
        batch_size=BATCH_SIZE,
        output_shapes=shapes,
        output_dtypes=dtypes,
        device_id=0)
    
    valid_data_set = dali_tf.DALIDataset(
        pipeline=Pipeline_valid,
        batch_size=BATCH_SIZE,
        output_shapes=shapes,
        output_dtypes=dtypes,
        device_id=0)
    
    resnet_model = ResNet50(weights = 'imagenet', include_top=False, input_shape = (224,224,3))

    for layer in resnet_model.layers:
        layer.trainable = False
    
    model = tf.keras.models.Sequential([
                                    resnet_model,
                                    tf.keras.layers.GlobalMaxPooling2D(),
                                    tf.keras.layers.Dense(HIDDEN_SIZE, activation='relu'),
                                    tf.keras.layers.Dropout(DROPOUT),
                                    tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

94773248/94765736 [==============================] - 1s 0us/step


In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_max_pooling2d (Global (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 102)               52326     
Total params: 24,689,126
Trainable params: 1,101,414
Non-trainable params: 23,587,712
_________________________________________________________________


In [12]:
with tf.device('/gpu:0'):
    model.fit(data_set, epochs=EPOCHS, steps_per_epoch=ITERATIONS_PER_EPOCH,  validation_data = valid_data_set, validation_steps=VALIDATION_STEPS)

Epoch 1/20
102/102 [==============================] - 47s 368ms/step - loss: 5.1011 - accuracy: 0.0248 - val_loss: 4.4589 - val_accuracy: 0.0499
Epoch 2/20
102/102 [==============================] - 37s 367ms/step - loss: 4.4489 - accuracy: 0.0433 - val_loss: 4.3603 - val_accuracy: 0.0591
Epoch 3/20
102/102 [==============================] - 38s 372ms/step - loss: 4.3481 - accuracy: 0.0540 - val_loss: 4.2836 - val_accuracy: 0.0654
Epoch 4/20
102/102 [==============================] - 38s 370ms/step - loss: 4.2929 - accuracy: 0.0590 - val_loss: 4.2334 - val_accuracy: 0.0666
Epoch 5/20
102/102 [==============================] - 38s 371ms/step - loss: 4.2368 - accuracy: 0.0656 - val_loss: 4.1801 - val_accuracy: 0.0833
Epoch 6/20
102/102 [==============================] - 38s 371ms/step - loss: 4.2006 - accuracy: 0.0747 - val_loss: 4.1472 - val_accuracy: 0.0787
Epoch 7/20
102/102 [==============================] - 38s 371ms/step - loss: 4.1620 - accuracy: 0.0702 - val_loss: 4.0813 - val_ac

In [13]:
train_dataset = image_dataset_from_directory(train_directory,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=(IMAGE_SIZE, IMAGE_SIZE))

valid_dataset = image_dataset_from_directory(valid_directory,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=(IMAGE_SIZE, IMAGE_SIZE))

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
valid_dataset = valid_dataset.prefetch(buffer_size=AUTOTUNE)

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Normalization(mean=0., variance=255.),
  tf.keras.layers.experimental.preprocessing.RandomRotation((-10.0, 10.0)),
  tf.keras.layers.experimental.preprocessing.RandomFlip(),
  tf.keras.layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
])

Found 6552 files belonging to 102 classes.
Found 818 files belonging to 102 classes.


In [14]:
with tf.device('/gpu:0'):
    model.fit(train_dataset, epochs=EPOCHS, validation_data = valid_dataset)

Epoch 1/20
103/103 [==============================] - 26s 230ms/step - loss: 7.4929 - accuracy: 0.0736 - val_loss: 3.4394 - val_accuracy: 0.1870
Epoch 2/20
103/103 [==============================] - 23s 215ms/step - loss: 3.4931 - accuracy: 0.1557 - val_loss: 2.9158 - val_accuracy: 0.2787
Epoch 3/20
103/103 [==============================] - 24s 217ms/step - loss: 3.0483 - accuracy: 0.2291 - val_loss: 2.5191 - val_accuracy: 0.3619
Epoch 4/20
103/103 [==============================] - 24s 218ms/step - loss: 2.6910 - accuracy: 0.2886 - val_loss: 2.1938 - val_accuracy: 0.4315
Epoch 5/20
103/103 [==============================] - 23s 217ms/step - loss: 2.4811 - accuracy: 0.3275 - val_loss: 2.0609 - val_accuracy: 0.4523
Epoch 6/20
103/103 [==============================] - 23s 217ms/step - loss: 2.3227 - accuracy: 0.3643 - val_loss: 1.8694 - val_accuracy: 0.4817
Epoch 7/20
103/103 [==============================] - 23s 216ms/step - loss: 2.2163 - accuracy: 0.3916 - val_loss: 1.7704 - val_ac